In [11]:
import pandas as pd
import numpy as np

#calcula a entropia do conjunto de dados
def calculate_entropy(dataframe, target, decimal_places=4):
    entropy = 0
    for target_value in sorted(dataframe[target].unique()):
        target_value_count = dataframe[target].value_counts()[target_value]
        target_value_probability = target_value_count / len(dataframe)
        entropy += -target_value_probability * np.log2(target_value_probability)
    return round(entropy, decimal_places)

#calcula o ganho de informação de um atributo
def calculate_information_gain(dataframe, target, attribute, verbose=False, decimal_places=4):
    information_gain = calculate_entropy(dataframe, target)
    for attribute_value in sorted(dataframe[attribute].unique()):
        if verbose:
            print('attribute: {}'.format(attribute_value))
        attribute_value_count = dataframe[attribute].value_counts()[attribute_value]
        if verbose:
            print('total occurrences: {}'.format(attribute_value_count))
        attribute_value_probability = attribute_value_count / len(dataframe)
        if verbose:
            print('probability: {}'.format(attribute_value_probability))
        information_gain -= attribute_value_probability * calculate_entropy(dataframe[dataframe[attribute] == attribute_value], target) 
        if verbose:
            print('entropy: {}'.format(calculate_entropy(dataframe[dataframe[attribute] == attribute_value], target)))
            print('')

    if verbose:
        print('information gain: {}'.format(information_gain))
    
    return round(information_gain, decimal_places)

In [12]:
question_dict = {
    "question_1": "Se um de nós pedir desculpas quando nossa discussão piorar, a discussão termina.",
    "question_2": "Sei que podemos ignorar nossas diferenças, mesmo que as coisas fiquem difíceis às vezes.",
    "question_3": "Quando precisamos, podemos recomeçar nossas discussões com meu cônjuge e corrigi-las.",
    "question_4": "Quando discuto com meu cônjuge, entrar em contato com ele eventualmente funcionará.",
    "question_5": "O tempo que passo com minha esposa é especial para nós.",
    "question_6": "Não temos tempo em casa como parceiros.",
    "question_7": "Somos como dois estranhos que compartilham o mesmo ambiente em casa, em vez de uma família.",
    "question_8": "Eu aproveito nossas férias com minha esposa.",
    "question_9": "Eu gosto de viajar com minha esposa.",
    "question_10": "A maioria de nossos objetivos é comum ao meu cônjuge.",
    "question_11": "Acredito que um dia, olhando para trás, verei que meu cônjuge e eu estivemos em harmonia um com o outro.",
    "question_12": "Meu cônjuge e eu temos valores semelhantes em relação à liberdade pessoal.",
    "question_13": "Meu cônjuge e eu temos um senso de entretenimento semelhante.",
    "question_14": "A maioria de nossos objetivos em relação a pessoas (filhos, amigos, etc.) é a mesma.",
    "question_15": "Nossos sonhos com meu cônjuge são semelhantes e harmoniosos.",
    "question_16": "Somos compatíveis com meu cônjuge sobre o que o amor deve ser.",
    "question_17": "Compartilhamos as mesmas opiniões sobre ser feliz em nossa vida com meu cônjuge.",
    "question_18": "Meu cônjuge e eu temos ideias semelhantes sobre como o casamento deve ser.",
    "question_19": "Meu cônjuge e eu temos ideias semelhantes sobre como os papéis devem ser no casamento.",
    "question_20": "Meu cônjuge e eu temos valores semelhantes em relação à confiança.",
    "question_21": "Sei exatamente do que minha esposa gosta.",
    "question_22": "Sei como meu cônjuge quer ser cuidado quando está doente.",
    "question_23": "Sei a comida favorita do meu cônjuge.",
    "question_24": "Posso dizer qual tipo de estresse meu cônjuge está enfrentando em sua vida.",
    "question_25": "Tenho conhecimento do mundo interior do meu cônjuge.",
    "question_26": "Sei quais são as ansiedades básicas do meu cônjuge.",
    "question_27": "Sei quais são as fontes atuais de estresse do meu cônjuge.",
    "question_28": "Sei quais são as esperanças e desejos do meu cônjuge.",
    "question_29": "Conheço muito bem meu cônjuge.",
    "question_30": "Conheço os amigos do meu cônjuge e seus relacionamentos sociais.",
    "question_31": "Sinto-me agressivo quando discuto com meu cônjuge.",
    "question_32": "Ao discutir com meu cônjuge, geralmente uso expressões como 'você sempre' ou 'você nunca'.",
    "question_33": "Posso fazer declarações negativas sobre a personalidade do meu cônjuge durante nossas discussões.",
    "question_34": "Posso usar expressões ofensivas durante nossas discussões.",
    "question_35": "Posso insultar meu cônjuge durante nossas discussões.",
    "question_36": "Posso ser humilhante quando discutimos.",
    "question_37": "Minha discussão com meu cônjuge não é calma.",
    "question_38": "Odeio a maneira como meu cônjuge abre um assunto.",
    "question_39": "Nossas discussões frequentemente ocorrem de repente.",
    "question_40": "Estamos apenas começando uma discussão antes que eu saiba o que está acontecendo.",
    "question_41": "Quando falo com meu cônjuge sobre algo, minha calma quebra repentinamente.",
    "question_42": "Quando discuto com meu cônjuge, simplesmente saio e não digo uma palavra.",
    "question_43": "Na maioria das vezes, fico em silêncio para acalmar um pouco o ambiente.",
    "question_44": "Às vezes, acho que é bom para mim sair de casa por um tempo.",
    "question_45": "Prefiro ficar em silêncio do que discutir com meu cônjuge.",
    "question_46": "Mesmo que eu esteja certo na discussão, fico em silêncio para magoar meu cônjuge.",
    "question_47": "Quando discuto com meu cônjuge, fico em silêncio porque tenho medo de não conseguir controlar minha raiva.",
    "question_48": "Sinto que estou certo em nossas discussões.",
    "question_49": "Não tenho nada a ver com o que fui acusado.",
    "question_50": "Na verdade, não sou o culpado pelo que estou sendo acusado.",
    "question_51": "Não sou o errado em problemas em casa.",
    "question_52": "Não hesitaria em dizer ao meu cônjuge sobre sua inadequação.",
    "question_53": "Quando discuto, lembro meu cônjuge de sua inadequação.",
    "question_54": "Não tenho medo de dizer ao meu cônjuge sobre sua incompetência."
}

In [13]:
divorce_dataset = pd.read_csv('dataset/divorce.csv', sep=';')

#estatística descritiva: contagem, média, desvio padrão, mínimo, máximo e os quartis
divorce_dataset.describe()

,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9,question_10,...,question_46,question_47,question_48,question_49,question_50,question_51,question_52,question_53,question_54,class
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,1.776471,1.652941,1.764706,1.482353,1.541176,0.747059,0.494118,1.452941,1.458824,1.576471,...,2.552941,2.270588,2.741176,2.382353,2.429412,2.476471,2.517647,2.241176,2.011765,0.494118
std,1.627257,1.468654,1.415444,1.504327,1.632169,0.904046,0.898698,1.546371,1.557976,1.421529,...,1.371786,1.586841,1.137348,1.511587,1.405090,1.260238,1.476537,1.505634,1.667611,0.501442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,...,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,0.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,3.000000,3.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


In [14]:
#nome dos atributos
attributes_name_list = divorce_dataset.columns.tolist()[0:-1]

#nome do atributo meta
meta_attribute = divorce_dataset.columns.tolist()[-1]

In [15]:
#entropia do conjunto de dados
print(calculate_entropy(divorce_dataset, meta_attribute))

0.9999


In [16]:
#calcula o ganho de informação de cada atributo
gain_list = []

for attribute in attributes_name_list:
    gain_list.append(calculate_information_gain(divorce_dataset,meta_attribute,attribute))

gain_df = pd.DataFrame({'attribute': attributes_name_list, 'gain': gain_list})
gain_df.sort_values(by='gain', ascending=False, inplace=True, ignore_index=True)
gain_df['question'] = [question_dict[x] for x in gain_df['attribute']]
#top 10 atributos com maior ganho de informação
gain_df.head(10)

,attribute,gain,question
0,question_18,0.9119,Meu cônjuge e eu temos ideias semelhantes sobr...
1,question_20,0.8970,Meu cônjuge e eu temos valores semelhantes em ...
2,question_40,0.8900,Estamos apenas começando uma discussão antes q...
3,question_17,0.8821,Compartilhamos as mesmas opiniões sobre ser fe...
4,question_19,0.8715,Meu cônjuge e eu temos ideias semelhantes sobr...
5,question_11,0.8683,"Acredito que um dia, olhando para trás, verei ..."
6,question_9,0.8545,Eu gosto de viajar com minha esposa.
7,question_16,0.8369,Somos compatíveis com meu cônjuge sobre o que ...
8,question_15,0.8232,Nossos sonhos com meu cônjuge são semelhantes ...
9,question_26,0.8223,Sei quais são as ansiedades básicas do meu côn...


In [19]:
for index, row in gain_df.head(3).iterrows():
    print(row['question']+':'+str(row['gain'])+'\n')

    calculate_information_gain(divorce_dataset,meta_attribute,row['attribute'],verbose=True)

    print('\n ---------------- \n')


Meu cônjuge e eu temos ideias semelhantes sobre como o casamento deve ser.:0.9119

attribute: 0
total occurrences: 79
probability: 0.4647058823529412
entropy: 0.0979

attribute: 1
total occurrences: 10
probability: 0.058823529411764705
entropy: 0.7219

attribute: 2
total occurrences: 16
probability: 0.09411764705882353
entropy: 0.0

attribute: 3
total occurrences: 44
probability: 0.25882352941176473
entropy: 0.0

attribute: 4
total occurrences: 21
probability: 0.12352941176470589
entropy: 0.0

information gain: 0.9119405882352941

 ---------------- 

Meu cônjuge e eu temos valores semelhantes em relação à confiança.:0.897

attribute: 0
total occurrences: 81
probability: 0.4764705882352941
entropy: 0.096

attribute: 1
total occurrences: 10
probability: 0.058823529411764705
entropy: 0.971

attribute: 2
total occurrences: 21
probability: 0.12352941176470589
entropy: 0.0

attribute: 3
total occurrences: 36
probability: 0.21176470588235294
entropy: 0.0

attribute: 4
total occurrences: 22
pr